## Uncertainty estimation for annual irrigated area estimates

Uncertainty calculation on area estimates following _Olofsson et al. 2013 Making better use of accuracy data in land change studies._
This script requres the input of a confusion matrix that contains labelled classes and the map area of each class (in Ha)

In [1]:
import numpy as np
import pandas as pd

In [2]:
confusion_matrix = 'results/nmdb_plots/uncertainty/test_uncertainty.csv'
year = '1989-90'

In [4]:
a = pd.read_csv(confusion_matrix, index_col='Class')
a

,irrigated,non_irrigated,total,map_area,Wi
Class,,,,,
irrigated,245,5,250,21414355,0.020548
non_irrigated,1,249,250,1020763896,0.979452
total,246,254,500,1042178251,1.000000


In [7]:
#calculate 'Wi', the proportion of each clas
a['map_area'] = a['map_area']*(5*5) / 10000

In [8]:
def areaUncertainty(df):
    """
    Function that implements equations 1 through 5 in Olofsson et al. 2013 to
    recalculate area with an uncertainty bound
    """
    #adjusted area estimate to account for commission/omission
    stratified_estimate = a['Wi'][0]*(a['irrigated'][0]/a['total'][0])+a['Wi'][1]*(a['irrigated'][1]/a['total'][1])
    adjusted_area = a['map_area'][-1] * stratified_estimate
    #standard error
    i = round(a['Wi'][0], 6)**2 * (a['irrigated'][0]/a['total'][0] * (1-a['irrigated'][0]/a['total'][0]) / int(a['total'][0]-1))
    j = round(a['Wi'][1], 6)**2 * (a['irrigated'][1]/a['total'][1] * (1-a['irrigated'][1]/a['total'][1]) / int(a['total'][1]-1))
    standard_error = (i + j)**(0.5)
    SE_of_adjustedArea = a['map_area'][-1] * standard_error
    #confidence interval
    CI = 2 * SE_of_adjustedArea
    return adjusted_area, CI

adjusted_area, CI = areaUncertainty(a)

In [10]:
print("Irrigated area estimate for " +year +", with 95% confidence interval is " + str(round(adjusted_area, 2)) + " +- " + str(round(CI,2)) + " Ha")

Irrigated area estimate for 1989-90, with 95% confidence interval is 62672.81 +- 20437.36 Ha
